In [1]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [2]:
# Chemins vers les dossiers de données d'entraînement et de test
train_data_directory = 'extract/data'
test_data_directory = 'D:/projet python/FAA_project/new styve/test'
emotions = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]

In [3]:

# Prétraitement des données
def preprocess_data(data_directory):
    images = []
    labels = []

    for emotion in emotions:
        emotion_path = os.path.join(data_directory, emotion)
        for image_name in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_name)
            image = cv2.imread(image_path, 0)  # Charger l'image en niveaux de gris
            images.append(image)
            labels.append(emotions.index(emotion))  # Étiqueter l'image avec l'indice de l'émotion

    return np.array(images), np.array(labels)

# Chargement des données d'entraînement
train_images, train_labels = preprocess_data(train_data_directory)

# Chargement des données de test
test_images, test_labels = preprocess_data(test_data_directory)

In [4]:

# Extraction des caractéristiques
def extract_features(images):
    features = []

    for image in images:
        # À titre d'exemple, nous allons simplement à plat les images en vecteurs
        feature = image.flatten()
        features.append(feature)

    return np.array(features)

# Extraction des caractéristiques pour les données d'entraînement
train_features = extract_features(train_images)

# Extraction des caractéristiques pour les données de test
test_features = extract_features(test_images)

In [5]:

# Équilibrage des données d'entraînement avec SMOTE
smote = SMOTE()
train_features, train_labels = smote.fit_resample(train_features, train_labels)

In [6]:

# Construction du modèle
model = RandomForestClassifier()

In [7]:

# Entraînement du modèle avec K-fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
train_accuracies = []
test_accuracies = []

for train_index, test_index in kfold.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    model.fit(X_train, y_train)

    train_predictions = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_predictions)
    train_accuracies.append(train_accuracy)

    test_predictions = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)
    test_accuracies.append(test_accuracy)

In [8]:

# Évaluation du modèle sur les données de test
predictions = model.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
print("Précision sur les données de test :", accuracy)

print("Précision moyenne sur les données d'entraînement :", np.mean(train_accuracies))
print("Précision moyenne sur les données de test :", np.mean(test_accuracies))

Précision sur les données de test : 0.3388461172892311
Précision moyenne sur les données d'entraînement : 0.9996835134871945
Précision moyenne sur les données de test : 0.6319407926769889


In [9]:
import joblib

# Sauvegarde du modèle entraîné
joblib.dump(model, 'trained_model_forest.pkl')
print("Modèle sauvegardé avec succès.")

Modèle sauvegardé avec succès.
